In [1]:
folder = 'Cameras_new8'

from data_util.product import *
from data_util.mainCat import *
from data_util.mixCat import *
from data_util.MongoDB import *
from data_util.stopwords import *
from data_util.preprocess import *

In [2]:
def make_corpus():
    corpus_path = '%s/corpus.txt'%(folder) 
    corpus = []
    embedding_corpus = []
    with open(corpus_path,'r',encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = squeeze2(line)
            tokens = line.replace("\n",'').replace("<s>",'').replace("</s>",'').split(" ")
            # tokens = [token for token in tokens if (token != '' and token.isalpha() and token not in alphbet_stopword)]
            tokens = [token for token in tokens if (token != '' )] # => 最佳
            corpus.append(tokens)
            embedding_corpus.append(tokens)
    print('make corpus finished...')
    return embedding_corpus
embedding_corpus = make_corpus()

make corpus finished...


In [3]:
from glove import Glove
from glove import Corpus
from gensim import corpora
#Creating a corpus object

'''Using Corpus to construct co-occurrence matrix'''
corpus = Corpus() # 建立glove corpus物件，並設定matrix scan window大小
corpus.fit(embedding_corpus, window=10) 

In [4]:
'''
creating a Glove object which will use the matrix created in the above lines to create embeddings
We can set the learning rate as it uses Gradient Descent and number of components
'''

# 建立glove物件，並使用先前建立的co-occurrence matrix去建立embedding
glove = Glove(no_components=300, learning_rate=0.05, max_count=100)
glove.fit(corpus.matrix, epochs=100,
          no_threads=10, verbose=True)

'''
After the training glove object has the word vectors for the lines we have provided. But the dictionary still resides in the corpus object.
We need to add the dictionary to the glove object to make it complete

cfs : dict of (int, int)
|      Collection frequencies: token_id -> how many instances of this token are contained in the documents.
|  dfs : dict of (int, int)
|      Document frequencies: token_id -> how many documents contain this token.
'''

# 將glove物件加入dictionary，使word vectors與其匹配
glove.add_dictionary(corpus.dictionary)

Performing 100 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [5]:
import os

glove.word_vectors.shape

glove.save('%s/Embedding/glove/test/glove.model'%(folder)) # 存模型
corpus.save('%s/Embedding/glove/test/glove.corpus'%(folder)) # 存字典

glove = Glove.load('%s/Embedding/glove/test/glove.model'%(folder))
   
# write vocab to file
vocab_file = "%s/Embedding/glove/test/word.vocab"%(folder)
if not os.path.exists(vocab_file):
    vocab_count = len(glove.dictionary.keys()) 
    vocab_count = 0
    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word, word_id in glove.dictionary.items():
#             freq = 
            writer.write(word + ' ' + str(word_id) + '\n') # Output vocab count                    
    print("Finished writing vocab file %s" %(vocab_count))      

Writing vocab file...
Finished writing vocab file 0


In [6]:
freq_dictionary = corpora.Dictionary(embedding_corpus)
freq_dict = {}

for word, word_id in glove.dictionary.items():
    freq = freq_dictionary.cfs[freq_dictionary.token2id[word]]
    freq_dict[word] = freq

In [7]:
import torch as T
import numpy as np
glove = Glove.load('%s/Embedding/glove/test/glove.model'%(folder))
weight = T.zeros(50000, 300)
for i in range(4):
    weight[i, :] = T.from_numpy(np.random.normal(0, 1e-4, 300))

for word,wid in glove.dictionary.items():
    try:
        vector = np.asarray(glove.word_vectors[wid], "float32")
        if wid + 4 >= len(weight): break
        weight[wid + 4, :] = T.from_numpy(vector)
    except Exception:
        print("no indexes %s" % wid)
        continue
        
embeds = T.nn.Embedding.from_pretrained(weight)
embeds.weight.requires_grad = True

In [8]:
embeds.weight[-1]

tensor([-0.0268,  0.1907, -0.0251,  0.1371,  0.0110, -0.0868,  0.0071,  0.0517,
         0.0615, -0.0614,  0.0339,  0.0823, -0.0534, -0.0316, -0.0499, -0.2044,
         0.0208, -0.0113, -0.0532, -0.1295,  0.0134, -0.1156,  0.1618, -0.0787,
         0.0357, -0.0870, -0.0029,  0.0042,  0.1614,  0.0598, -0.0582, -0.1131,
        -0.0509,  0.0581, -0.1741, -0.1129,  0.0122, -0.0233, -0.0813, -0.0405,
         0.0462, -0.0070,  0.0088,  0.0106,  0.0466, -0.1232,  0.1140,  0.0129,
         0.0476, -0.1354, -0.0433, -0.0798,  0.1246,  0.2255, -0.0053, -0.0683,
        -0.0504, -0.1818, -0.0135,  0.2014, -0.0506, -0.0174, -0.0418, -0.0699,
         0.0360,  0.1954, -0.0902,  0.0430,  0.0232,  0.0434,  0.0259,  0.0082,
         0.0531, -0.0354, -0.1017, -0.1919, -0.0501, -0.1852, -0.1067, -0.0252,
        -0.2079,  0.0046, -0.0683,  0.0155,  0.1655, -0.0934,  0.3866, -0.0261,
         0.0410, -0.0303,  0.0123,  0.0794,  0.0236,  0.0444,  0.0447, -0.0667,
        -0.0118,  0.0101, -0.0145, -0.06